# WordCount avec GCP et PySpark

__Auteur__: Kiril Isakov  
__Date de création__: 2023-07-25  
__Présentation__: Ce notebook permet de compter le nombre d'occurence de chaque mot dans un text. Il utilise pour cela 
pyspark.

__Prérequis__: Un bucket gcp pour le stockage des données. (BUCKET_NAME)  
__Inputs__: texte d'entrée: `gs://{BUCKET_NAME}/input/{INPUT_FILE_NAME}`  
__Outputs__: csv de sortie: `gs://{BUCKET_NAME}/output/{OUTPUT_FILE_NAME}`

__Params__:
- `BUCKET_NAME`: nom du bucket GCP. DOIT être dans le même projet.
- `INPUT_FILE_NAME`: nom du fichier a traiter. DOIT être au format "txt"
- `OUTPUT_FILE_NAME`: nom du fichier de sortie contenant le résulta.

### Exemple de lecture d'un fichier depuis un bucket GCP avec Python

In [15]:
BUCKET_NAME = 'data_m2i'
PATH_TO_DATA = 'input/exemple.txt'

In [16]:
from google.cloud import storage
import pandas as pd

In [17]:
client = storage.Client()
bucket = client.bucket(BUCKET_NAME)

In [18]:
file_blob = bucket.blob(PATH_TO_DATA)
for blob in client.list_blobs(BUCKET_NAME):
    print(blob.name)

input/
input/exemple.txt
output/


In [19]:
with file_blob.open(mode='r') as f:
    data_as_str = f.read()

In [20]:
data_as_str[:300]

'Ceci est un fichier exemple\nIl ne contient rien de très utile\nCet exemple peut etre remplacer par un autre\nDans les couloirs de l hôpital des punks passent\nDes ministres marinent à proximité en cuisinant\nJ ai peine à ne guère compromettre les vaisseaux\nGuy Bedos parie sur une papille\nLa remplaçante '

### Même exemple avec PySpark

In [21]:
file_uri = f'gs://{BUCKET_NAME}/{PATH_TO_DATA}'
text_file = sc.textFile(file_uri)
data_as_list = text_file.collect()

23/07/25 12:01:54 WARN GhfsStorageStatistics: Detected potential high latency for operation op_glob_status. latencyMs=134; previousMaxLatencyMs=110; operationCount=3; context=path=gs://data_m2i/input/exemple.txt; pattern=org.apache.hadoop.mapred.FileInputFormat$MultiPathFilter@20143668


In [22]:
data_as_list[:8]

['Ceci est un fichier exemple',
 'Il ne contient rien de très utile',
 'Cet exemple peut etre remplacer par un autre',
 'Dans les couloirs de l hôpital des punks passent',
 'Des ministres marinent à proximité en cuisinant',
 'J ai peine à ne guère compromettre les vaisseaux',
 'Guy Bedos parie sur une papille',
 'La remplaçante dit adieu à tous les colporteurs']

### WordCount avec Python

In [23]:
def remove_punctuation(s: str):
    return ''.join(filter(lambda x: x.isalnum() or x in ['-', "'"], s))

In [24]:
words = []
for line in data_as_list:
    for word_dirty in line.split(' '):
        word_clean = remove_punctuation(word_dirty)
        words.append(word_clean.lower())

In [25]:
word_count = {}
for word in words:
    if word in word_count.keys():
        word_count[word] += 1
    else:
        word_count[word] = 1

In [26]:
data = {'word': word_count.keys(), 'count': word_count.values()}
pd.DataFrame.from_dict(data).sort_values(by='count', ascending=False)

,word,count
23,des,56
28,à,55
9,de,54
2,un,44
19,les,36
...,...,...
306,occupent,1
307,athée,1
308,gouverneur,1
310,mine,1


### WordCount avec PySpark

In [27]:
file_uri = f'gs://{BUCKET_NAME}/{PATH_TO_DATA}'
text_file_rdd = sc.textFile(file_uri)
words_rdd = text_file_rdd.flatMap(lambda line: line.split(' '))
word_count_rdd = words_rdd.map(lambda word: (remove_punctuation(word).lower(), 1))\
                          .reduceByKey(lambda x, y: (x + y))\
                          .sortBy(lambda x: x[1], ascending=False)

In [28]:
word_count_rdd.take(10)

[('des', 56),
 ('à', 55),
 ('de', 54),
 ('un', 44),
 ('les', 36),
 ('la', 31),
 ('le', 29),
 ('l', 23),
 ('nous', 20),
 ('d', 18)]